# Code for bachelor's thesis
## Data preprocessing
### Normalization

In [ ]:
import pandas as pd
import numpy as np

# Read the CSV file
df = pd.read_csv('new_baseline.csv')

sqrt = ['population_density', 'main_road_share', 'car_density', 'traff_score']
log = ['bus_stops', 'main_road_density', 'all_road_density', 'bus_frequency', 'separated_frequency', 'bus_saturation', 'cars_per_road', 'separated_stops', 'bus_stop_density', 'separated_stop_density', 'cars_and_buses', 'population']

# Apply square root transformation to selected columns
for column in df.columns:
    if column in sqrt:
        df[column] = np.sqrt(df[column])
        
for column in df.columns:
    if column in log:
        df[column] = np.log(df[column])
        
df.to_csv('new_normed.csv', index = False)

### Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Load data into a pandas DataFrame
df = pd.read_csv('new_normed.csv')

# Initialize the scaler
scaler = MinMaxScaler()

# Scale the data
scaled_data = scaler.fit_transform(df)

# Create a new DataFrame with the scaled data
scaled_df = pd.DataFrame(scaled_data, columns=df.columns)

# Save the scaled data to a new CSV file
scaled_df.to_csv('new_scaled.csv', index=False)

## Model creation
### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

# Load data into a pandas DataFrame
df = pd.read_csv('new_scaled.csv')

# Get a list of all variable names
variable_names = df.columns.tolist()

# Initialize empty DataFrame to store regression coefficients
coefficients_df = pd.DataFrame(columns=variable_names, index=variable_names)

# Loop through each pair of variables and run regression both ways
for i in range(len(variable_names)):
    for j in range(i+1, len(variable_names)):
        var1 = variable_names[i]
        var2 = variable_names[j]
        # Create a list of all other variables in the dataset
        other_vars = [v for v in variable_names if v not in [var1, var2]]
        # Create a LinearRegression object and fit the data
        model = LinearRegression().fit(df[[var2] + other_vars], df[var1])
        # Store the regression coefficient for the independent variable
        coefficients_df.loc[var1, var2] = model.coef_[0]
        
            # Create a LinearRegression object and fit the data
        model = LinearRegression().fit(df[[var1] + other_vars], df[var2])
        # Store the regression coefficient for the independent variable
        coefficients_df.loc[var2, var1] = model.coef_[0]

# Print the regression coefficients as a matrix
print(coefficients_df)

coefficients_df.to_csv('linear_weights.csv', index = False)


### GlassoFCM

In [ ]:
from sklearn.covariance import GraphicalLasso
import networkx as nx

def glassoFCM(data, alpha):
    # Initialize the GraphicalLasso model
    model = GraphicalLasso(alpha=alpha, max_iter=1000, mode = 'cd')
    
    # Fit the model to the data
    model.fit(data)
    
    # Get the inverse covariance matrix (precision matrix)
    precision_matrix = model.precision_
    
    # Since we are only interested in the weights (and not the directions of the edges), 
    # we can convert the precision matrix to a correlation matrix
    correlation_matrix = precision_matrix / np.sqrt(np.outer(np.diag(precision_matrix), np.diag(precision_matrix)))
    
    # The weights in the FCM are given by the elements of the correlation matrix
    weights = correlation_matrix
    
    return weights

def invert_order(order):
    # Get the inverted order
    inverted_order = np.argsort(order)

    return inverted_order

def calculate_centrality_measures(weights):
    # Create a directed graph from the weight matrix
    G = nx.from_numpy_matrix(weights, create_using=nx.DiGraph)
    
    # Calculate the various centrality measures
    strength_centrality = np.array(list(nx.degree_centrality(G).values()))
    closeness_centrality = np.array(list(nx.closeness_centrality(G).values()))
    betweenness_centrality = np.array(list(nx.betweenness_centrality(G).values()))
    
    return strength_centrality, closeness_centrality, betweenness_centrality

def reorder_matrix(weights, order):
    # Reorder the rows and columns of the weight matrix
    weights_reordered = weights[order][:, order]
    
    return weights_reordered

def glassoFCM_asymmetric(data, alpha):
    # Get the symmetric weights using the glassoFCM function
    weights = glassoFCM(data, alpha)
    
    # Calculate the various centrality measures
    strength_centrality, closeness_centrality, betweenness_centrality = calculate_centrality_measures(weights)
    
    # Get the order of the nodes based on each centrality measure
    order_strength = np.argsort(strength_centrality)
    order_closeness = np.argsort(closeness_centrality)
    order_betweenness = np.argsort(betweenness_centrality)    
    
    # Reorder the weight matrix based on each centrality measure
    weights_strength = reorder_matrix(weights, order_strength)
    weights_closeness = reorder_matrix(weights, order_closeness)
    weights_betweenness = reorder_matrix(weights, order_betweenness)
    
    # Get the upper triangular part of each reordered weight matrix
    weights_strength_upper = np.triu(weights_strength)
    weights_closeness_upper = np.triu(weights_closeness)
    weights_betweenness_upper = np.triu(weights_betweenness)
    
    # Get the inverted order for each centrality measure
    inverted_order_strength = invert_order(order_strength)
    inverted_order_closeness = invert_order(order_closeness)
    inverted_order_betweenness = invert_order(order_betweenness)
    
    # Reorder the upper triangular weight matrix to the original order
    weights_strength_final = reorder_matrix(weights_strength_upper, inverted_order_strength)
    weights_closeness_final = reorder_matrix(weights_closeness_upper, inverted_order_closeness)
    weights_betweenness_final = reorder_matrix(weights_betweenness_upper, inverted_order_betweenness)
    
    return weights_strength_final, weights_closeness_final, weights_betweenness_final

df = pd.read_csv('new_scaled.csv')

df['traffic_score'] = df['traff_score']
df.drop('traff_score', axis=1, inplace=True)

weights_strength, weights_closeness, weights_betweenness = glassoFCM_asymmetric(df.values, 0.01)
weights_strength.to_csv('glasso_weights.csv', index=False)

# Running the simulations

In [ ]:
import matplotlib.pyplot as plt

df_w = pd.read_csv('glasso_weights.csv')
df_w = df_w.fillna(0)

def transfer_function(raw_activation_vector):
    """Re-scaled transfer function"""
    norm = np.linalg.norm(raw_activation_vector)
    if norm == 0:
        return raw_activation_vector
    else:
        return raw_activation_vector / norm

def quasi_nonlinear_reasoning_rule(weights, raw_activation_vector, initial_values, phi):
    """Quasi Nonlinear Reasoning Rule"""
    new_activation_vector = []
    for i in range(len(weights)):
        temp = phi * np.dot(raw_activation_vector, weights[i]) + (1 - phi) * initial_values[i]
        new_activation_vector.append(temp)
    return np.array(new_activation_vector)

def simulate_FCM(weights, initial_values, phi, iterations):
    """Simulate FCM"""
    activation_vector = np.array(initial_values)
    activation_values_history = [activation_vector]
    for _ in range(iterations):
        raw_activation_vector = np.dot(activation_vector, weights)
        activation_vector = transfer_function(quasi_nonlinear_reasoning_rule(weights, raw_activation_vector, initial_values, phi))
        activation_values_history.append(activation_vector)
    return np.array(activation_values_history)

weight_matrix = df_w.values


# The names of your concepts
concept_names = ['population_density', 'main_road_share', 'motorization_rate',
       'car_density', 'all_road_density', 'traff_score', 'bus_frequency',
       'separated_frequency', 'bus_saturation', 'bus_stop_density',
       'separated_stop_density']

# Initialize storage for the steady states
steady_states = []

# Range of initial values for the first concept
init_values = np.arange(0.001, 1.1, 0.01)

df_i = pd.read_csv('initial.csv')

avg_activation_values = df_i.mean().values
avg_activation_values = df_i.iloc[14] #Tilburg:56, Brussel:0, Charleroi: 6, Zurich:14
init_activation_values = np.copy(avg_activation_values)

changed = 0

# Iterate over the range of initial values
for init_value in init_values:
    # Set the initial value of the first concept
    init_activation_values[changed] = init_value
    
    # Run the simulation
    activation_values = simulate_FCM(weight_matrix, init_activation_values, iterations=50, phi=0.1)
    
    # Store the steady state
    steady_states.append(activation_values[-1])

# Convert the list of steady states to a 2D numpy array
steady_states = np.array(steady_states)

# Create a plot for each concept
for i, concept_name in enumerate(concept_names):
    plt.figure(figsize=(10, 6))
    plt.plot(init_values, steady_states[:, i])
    plt.xlabel("Initial value of {}".format(concept_names[changed]))
    plt.ylabel("Steady state value of "+concept_name)
    plt.title("Change in steady state of "+concept_name)
    plt.tight_layout()
    if i == 5:
        plt.savefig('graphs/Zurich/{} v traff-score-zurich-glasso.pdf'.format(concept_names[changed]))
    plt.show()


# Create a plot for each concept
for i, concept_name in enumerate(concept_names):
    plt.plot(init_values, steady_states[:, i], label=concept_name)

plt.xlabel("Initial value of population_density")
plt.ylabel("Steady state value")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()
